In [9]:
import altair as alt
import pandas as pd

source = pd.read_excel('OECD_Data.xlsx')

# Convert the units row (row 1) into a dictionary
units = source.iloc[0].to_dict()

# Create a dictionary for simplified names of the units
simplified_units = {
    "Percentage": "%",
    "Ratio": "ratio",
    "US Dollar": "$",
    "Average score": "average score",
    "Years": "years",
    "Micrograms per cubic metre": "µg/m³",
    "Hours": "hours"
}

# Remove the units row from the source data
source = source.iloc[1:]

# Reset the DataFrame index
source.reset_index(drop=True, inplace=True)

# Add units to each of the indicators
for column in source.columns:
    unit = units[column]
    if unit in simplified_units:
        source.rename(columns={column: f"{column} ({simplified_units[unit]})"}, inplace=True)


df = source

# Melt the DataFrame to have columns: 'Country', 'Indicator', 'Value'
df = df.melt(id_vars='Country', var_name='Indicator', value_name='Value')

# Filter data for the two indicators
dwelling_data = df[df['Indicator'] == 'Dwellings without basic facilities (%)']
leisure_data = df[df['Indicator'] == 'Time devoted to leisure and personal care (hours)']

# Merge the two dataframes on 'Country'
merged_data = pd.merge(dwelling_data, leisure_data, on='Country')

# Rename columns for clarity
merged_data.columns = ['Country', 'Indicator_x', 'Dwelling%', 'Indicator_y', 'LeisureTime']

# Create the scatter plot
scatter_plot = alt.Chart(merged_data).mark_circle(size=60).encode(
    x='Dwelling%',
    y='LeisureTime',
    tooltip=['Country', 'Dwelling%', 'LeisureTime']
).interactive()

scatter_plot


c:\Users\Dom\Dev\Newcastle\CSC3833\W1\venv\lib\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [ ]:
# Select numeric columns and ignore 'Country' column
numeric_cols = indicators

print(numeric_cols)

# Apply manual Min-Max normalization to the numeric columns
for col in numeric_cols:
    source[col] = (source[col] - source[col].min()) / (source[col].max() - source[col].min())

# Melt the DataFrame into a format where one row is one data point
source_melted = source.melt(id_vars='Country')

# Create the heatmap
heatmap = alt.Chart(source_melted).mark_rect().encode(
    x='Country:N',
    y='variable:N',
    color='value:Q',
    tooltip=[
        alt.Tooltip('Country:N', title='Country'),
        alt.Tooltip('variable:N', title='Indicator'),
        alt.Tooltip('value:Q', title='Value')
    ]
).properties(width=600, height=300)

heatmap

In [11]:
import pandas as pd
import altair as alt

# Create dataframe
data = [[7, 10, 'Alex', 'Smith'],
        [12, 20, 'Bob', 'Jones'],
        [10, 30, 'Clive', 'Smith'],
        [42, 40,  'Alex', 'Johnson']]
df = pd.DataFrame(data,columns=['Favorite number', 'Age', 'First name', 'Last name'])

# Create selections
selection_first_name = alt.selection_multi(fields=['First name'], empty='none')
selection_last_name = alt.selection_multi(fields=['Last name'], empty='none')

# Create interactive scatter plot
scatter = alt.Chart(df).mark_point(size=100).encode(
    x='Favorite number:Q',
    y='Age:Q',
    color=alt.condition(selection_first_name & selection_last_name,
                        alt.Color('First name:N', legend=None),
                        alt.value('lightgray') ),
    shape=alt.Shape('Last name:N', legend=None),
    tooltip=['First name', 'Last name']
).add_selection(
    alt.selection_interval(bind='scales')
)

# Create interactive model name legend
legend_first_name = alt.Chart(df).mark_point(size=100).encode(
    y=alt.Y('First name:N', axis=alt.Axis(orient='right')),
    color=alt.condition(selection_first_name,
                        alt.Color('First name:N', legend=None),
                        alt.value('lightgray') ),
).add_selection(
    selection_first_name

)

# Create interactive model name legend
legend_last_name = alt.Chart(df).mark_point(size=100).encode(
    y=alt.Y('Last name:N', axis=alt.Axis(orient='right')),
    shape=alt.Shape('Last name:N', legend=None),
    color=alt.condition(selection_last_name,
                        alt.value('black'),
                        alt.value('lightgray') ),
).add_selection(
    selection_last_name
)

# Combine plotting elements
chart = scatter | legend_first_name | legend_last_name

chart

c:\Users\Dom\Dev\Newcastle\CSC3833\W1\venv\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\Dom\Dev\Newcastle\CSC3833\W1\venv\lib\site-packages\altair\vegalite\v5\api.py:369: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\Dom\Dev\Newcastle\CSC3833\W1\venv\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\Dom\Dev\Newcastle\CSC3833\W1\venv\lib\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.HConcatChart(...)

In [ ]:
indicator = "Life satisfaction (average score)"

choropleth = (
    alt.Chart(countries_shape)
    .mark_geoshape()
    .transform_lookup(
        lookup='NAME',
        from_=alt.LookupData(data=source, key='Country', fields=[indicator])
    )
    .encode(
        color=alt.Color(f"{indicator}:Q", legend=alt.Legend(
        orient='none',
        legendX=180, legendY=380,
        direction='horizontal',
        gradientLength=350)),
        opacity=alt.condition(click_countries, alt.value(1), alt.value(0.2)),
        tooltip=[
            alt.Tooltip('NAME:N', title='Country'),
            alt.Tooltip(f'{indicator}:Q')
        ]
    )
    .add_params(
        click_countries
    )
)

map = background + choropleth

chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('Country:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('y:Q', title=''),
    color=color_scale,
    tooltip='tooltip:N',
    opacity=alt.condition(click, alt.value(1), alt.value(0.7))
).properties(
    width=800,
    height=400
).transform_calculate(
    y=f'datum[{indicator_param.name}]' # Variable Y axis
).transform_calculate(
    tooltip='datum.Country + ": " + format(datum.y, ",")'
).transform_filter(
    'isValid(datum.y)' # Removes NaN values
).add_params(
    indicator_param # Add dropdown parameter
).add_params(
    click
)



# (chart | (basemap + map).project("equalEarth").properties(width=700, height=400))
(basemap + map).project("equalEarth").properties(width=700, height=400) | chart